In [141]:
import pandas as pd
import pulp

In [142]:
# Load data from CSV files
demand_data = pd.read_csv('./2_weekly_demand.csv')
sales_data = pd.read_csv('./3_sales.csv')
weekly_sales_data = pd.read_csv('./3_weekly_sales.csv')
transport_cost_data = pd.read_csv('./4_transport_cost.csv')
initial_inventory_data = pd.read_csv('./5_initial_inventory.csv')

In [143]:
# Create the optimization problem
prob = pulp.LpProblem("Inventory_Optimization", pulp.LpMinimize)

# Extract sets and parameters from the data
I = set(transport_cost_data['Supplier code'].unique())  # Set of suppliers
P = set(demand_data['Product'].unique())  # Set of products
T = range(len(demand_data))  # Assuming each row is a time period (week)

In [144]:
demand_data.columns

Index(['Product', 'Supplier', 'Supplier Country', 'Supplier City',
       'Lead Time (Weeks)', 'Mean Weekly Demand', 'Total_Sales',
       'Safety Stock (SS)', 'SS (round unit up)', 'SS (CBM)', 'SS (KG)',
       'Reorder Point (RP=s)', 'RP (round unit up)', 'RP (CBM)', 'RP (KG)',
       'Order-Up-To Level (S)', 'S (round unit up)', 'S (CBM)', 'S (KG)',
       'CBM/Orderlines', 'Kg/Orderlines'],
      dtype='object')

In [145]:
sales_data.columns

Index(['Delivery ID', 'Order ID', 'Delivery Priority', 'Division',
       'Distribution Channel', 'Sold-To Party', 'Ship-To Party',
       'Goods Issue Date', 'Supplier', 'Supplier Country', 'Supplier City',
       'Product', 'Orderlines', 'Net Weight (kg)', 'Volume (cdm)',
       'Sales (AUD)', 'Zone', 'Receiver Suburb', 'Receiver State',
       'Receiver Postcode'],
      dtype='object')

In [146]:
weekly_sales_data.columns

Index(['Product', 'Week No.', 'Weekly Sales', 'Total Sales',
       'Average Weekly Demand', 'Standard Deviation'],
      dtype='object')

In [147]:
transport_cost_data.columns

Index(['Supplier code', 'Supplier Country', 'City code', 'Mode', 'Cost',
       'Cost unit', 'Lead Time', 'Lead Time unit'],
      dtype='object')

In [148]:
transport_cost_data.head(10)

,Supplier code,Supplier Country,City code,Mode,Cost,Cost unit,Lead Time,Lead Time unit
0,CN,China,MEL,FCL20,6746.41,Container,40.000000,days
1,CN,China,MEL,FCL40,10925.46,Container,40.000000,days
2,CN,China,MEL,LCL,242.68,cubic meter,32.000000,days
3,CN,China,MEL,Air,11.79,kg of chargeable weight,10.083333,days
4,DE,Germany,MEL,FCL20,8355.55,Container,88.000000,days
5,DE,Germany,MEL,FCL40,13125.25,Container,88.000000,days
6,DE,Germany,MEL,LCL,597.87,cubic meter,65.000000,days
7,DE,Germany,MEL,Air,11.44,kg of chargeable weight,7.083333,days
8,MY,Malaysia,MEL,FCL20,6319.87,Container,47.000000,days
9,MY,Malaysia,MEL,FCL40,10197.50,Container,47.000000,days


In [149]:
initial_inventory_data.columns

Index(['Product', 'Supplier', 'Supplier City', 'Supplier Country',
       'Initial Lead time', 'First lot ordered', 'CBM', 'KG', 'CBM/Orderlines',
       'Kg/Orderlines'],
      dtype='object')

In [150]:
demand_data.drop(columns=['Lead Time (Weeks)', 'Mean Weekly Demand', 'Total_Sales', 'Safety Stock (SS)','SS (round unit up)', 'SS (CBM)', 'SS (KG)','Reorder Point (RP=s)', 'RP (round unit up)'])

,Product,Supplier,Supplier Country,Supplier City,RP (CBM),RP (KG),Order-Up-To Level (S),S (round unit up),S (CBM),S (KG),CBM/Orderlines,Kg/Orderlines
0,A-001,S-CN-001,China,Dongguan,2.303499,705.842472,5.682967,6,3.455248,1058.763708,0.575875,176.460618
1,A-002,S-CN-001,China,Dongguan,13.131633,2288.542818,175.387912,176,22.882846,3987.955801,0.130016,22.658840
2,A-003,S-CN-002,China,Shenzhen,11.136575,819.925778,120.419231,121,19.250366,1417.300274,0.159094,11.713225
3,A-004,S-DE-001,Germany,Mainz,38.999409,12075.321250,14536.650550,14537,77.239021,23915.387590,0.005313,1.645139
4,A-005,S-CN-003,China,Changzhou,215.344474,38375.952960,3969.488462,3970,411.413649,73316.907250,0.103631,18.467735
...,...,...,...,...,...,...,...,...,...,...,...,...
78,B-039,S-DE-004,Germany,Munich,5.075981,3468.919601,8.040110,9,9.136765,6244.055282,1.015196,693.783920
79,B-040,S-US-003,USA,Phoenix,0.228345,82.397518,0.652015,1,0.228345,82.397518,0.228345,82.397518
80,B-041,S-DE-005,Germany,Frankfurt,0.050189,11.424678,0.313187,1,0.050189,11.424678,0.050189,11.424678
81,B-042,S-DE-004,Germany,Munich,5.407723,2821.837760,16.093407,17,10.214587,5330.137991,0.600858,313.537529


In [151]:
# weekly_sales.head()
weekly_sales = pd.merge(demand_data[['Product', 'CBM/Orderlines']], weekly_sales_data, on='Product')

In [152]:
weekly_sales['CBM Sales'] = weekly_sales['CBM/Orderlines'] * weekly_sales['Weekly Sales']
weekly_sales.head(10)

,Product,CBM/Orderlines,Week No.,Weekly Sales,Total Sales,Average Weekly Demand,Standard Deviation,CBM Sales
0,A-001,0.575875,0,0,33,0.423077,0.67457,0.000000
1,A-001,0.575875,1,0,33,0.423077,0.67457,0.000000
2,A-001,0.575875,2,0,33,0.423077,0.67457,0.000000
3,A-001,0.575875,3,0,33,0.423077,0.67457,0.000000
4,A-001,0.575875,4,0,33,0.423077,0.67457,0.000000
5,A-001,0.575875,5,0,33,0.423077,0.67457,0.000000
6,A-001,0.575875,6,0,33,0.423077,0.67457,0.000000
7,A-001,0.575875,7,0,33,0.423077,0.67457,0.000000
8,A-001,0.575875,8,0,33,0.423077,0.67457,0.000000
9,A-001,0.575875,9,1,33,0.423077,0.67457,0.575875


In [153]:
merged_df = pd.merge(demand_data[["Supplier", "Supplier Country"]], transport_cost_data, on='Supplier Country')
merged_df = merged_df.drop_duplicates(subset=['Supplier', 'Mode'], keep='first')
merged_df.head(10)

,Supplier,Supplier Country,Supplier code,City code,Mode,Cost,Cost unit,Lead Time,Lead Time unit
0,S-CN-001,China,CN,MEL,FCL20,6746.41,Container,40.000000,days
1,S-CN-001,China,CN,MEL,FCL40,10925.46,Container,40.000000,days
2,S-CN-001,China,CN,MEL,LCL,242.68,cubic meter,32.000000,days
3,S-CN-001,China,CN,MEL,Air,11.79,kg of chargeable weight,10.083333,days
8,S-CN-002,China,CN,MEL,FCL20,6746.41,Container,40.000000,days
9,S-CN-002,China,CN,MEL,FCL40,10925.46,Container,40.000000,days
10,S-CN-002,China,CN,MEL,LCL,242.68,cubic meter,32.000000,days
11,S-CN-002,China,CN,MEL,Air,11.79,kg of chargeable weight,10.083333,days
12,S-DE-001,Germany,DE,MEL,FCL20,8355.55,Container,88.000000,days
13,S-DE-001,Germany,DE,MEL,FCL40,13125.25,Container,88.000000,days


In [154]:
merged_df[merged_df.duplicated(subset=['Supplier', 'Mode'], keep=False)]


,Supplier,Supplier Country,Supplier code,City code,Mode,Cost,Cost unit,Lead Time,Lead Time unit


In [155]:
import numpy as np


transport_cost_data = pd.read_csv('./4_transport_cost.csv')

transport_cost_data = pd.merge(demand_data[['Supplier', 'Supplier Country']], transport_cost_data, on='Supplier Country')
transport_cost_data = transport_cost_data.drop_duplicates(subset=['Supplier', 'Mode'], keep='first')
transport_cost_data['Lead Time (Weeks)'] = np.ceil(transport_cost_data['Lead Time'] / 7)

# transport_cost_fcl20 = transport_cost_data.set_index(['Supplier Country', 'Mode'])['Cost'].unstack().loc[:, 'FCL20']
# transport_cost_fcl20.loc[transport_cost_fcl20['Supplier' == 'S-CN-009', 'Lead Time (Weeks)']].values[0]

# transport_cost_fcl20.loc[transport_cost_fcl20['Supplier'] == 'S-CN-009', 'Lead Time (Weeks)'].values[0]
# type(transport_cost_data), type(demand_data)
transport_cost_data.loc[(transport_cost_data['Supplier'] == 'S-CN-009') & (transport_cost_data['Mode'] == 'FCL20'), 'Supplier'].values[0]

'S-CN-009'

In [156]:
sales_data.head()

,Delivery ID,Order ID,Delivery Priority,Division,Distribution Channel,Sold-To Party,Ship-To Party,Goods Issue Date,Supplier,Supplier Country,Supplier City,Product,Orderlines,Net Weight (kg),Volume (cdm),Sales (AUD),Zone,Receiver Suburb,Receiver State,Receiver Postcode
0,D5226436774112,O51406049681610,Standard Order,A,Retail,CA3019131146711,CA3119131146711,22/1/2021,S-DE-002,China,Changzhou,A-012,1,3.477964,4.253818,213.390895,MAC,MACKAY,QLD,4740
1,D5226442488812,O51406056273310,Standard Order,A,Omni Channel,CA3019131149011,CA3119131149011,4/1/2021,S-CN-003,China,Changzhou,A-005,1,34.435581,192.340685,1859.028528,VC1,CLYDE NORTH,VIC,3978
2,D5226442488812,O51406056273310,Standard Order,A,Omni Channel,CA3019131149011,CA3119131149011,4/1/2021,S-MY-002,Germany,Darmstadt,A-029,1,14.103547,37.705893,2621.374666,VC1,CLYDE NORTH,VIC,3978
3,D5226442488812,O51406056273310,Standard Order,A,Omni Channel,CA3019131149011,CA3119131149011,4/1/2021,S-DE-002,China,Changzhou,A-012,5,1.464758,8.086267,478.426926,VC1,CLYDE NORTH,VIC,3978
4,D5226442488812,O51406056273310,Standard Order,A,Omni Channel,CA3019131149011,CA3119131149011,4/1/2021,S-DE-002,China,Changzhou,A-012,5,4.185596,9.334321,350.657237,VC1,CLYDE NORTH,VIC,3978


In [157]:
initial_inventory_data.head()

,Product,Supplier,Supplier City,Supplier Country,Initial Lead time,First lot ordered,CBM,KG,CBM/Orderlines,Kg/Orderlines
0,A-001,S-CN-001,Dongguan,China,5,0,0.000000,0.000000,0.575875,176.460618
1,A-002,S-CN-001,Dongguan,China,5,0,0.000000,0.000000,0.130016,22.658840
2,A-003,S-CN-002,Shenzhen,China,5,68,10.818388,796.499328,0.159094,11.713225
3,A-005,S-CN-003,Changzhou,China,5,1676,173.684956,30951.923560,0.103631,18.467735
4,A-006,S-CN-003,Changzhou,China,5,1373,114.144326,13881.065250,0.083135,10.110026


In [158]:
products = demand_data['Product'].unique()
ignored_products = np.array(['B-037', 'B-038', 'B-041', 'B-043'])
products = np.setdiff1d(products, ignored_products)
products

array(['A-001', 'A-002', 'A-003', 'A-004', 'A-005', 'A-006', 'A-007',
       'A-008', 'A-009', 'A-010', 'A-011', 'A-012', 'A-013', 'A-014',
       'A-015', 'A-016', 'A-017', 'A-018', 'A-019', 'A-020', 'A-021',
       'A-022', 'A-023', 'A-024', 'A-025', 'A-026', 'A-027', 'A-028',
       'A-029', 'A-030', 'A-031', 'A-032', 'A-033', 'A-034', 'A-035',
       'A-036', 'A-037', 'A-038', 'A-039', 'A-040', 'B-001', 'B-002',
       'B-003', 'B-004', 'B-005', 'B-006', 'B-007', 'B-008', 'B-009',
       'B-010', 'B-011', 'B-012', 'B-013', 'B-014', 'B-015', 'B-016',
       'B-017', 'B-018', 'B-019', 'B-020', 'B-021', 'B-022', 'B-023',
       'B-024', 'B-025', 'B-026', 'B-027', 'B-028', 'B-029', 'B-030',
       'B-031', 'B-032', 'B-033', 'B-034', 'B-035', 'B-036', 'B-039',
       'B-040', 'B-042'], dtype=object)

In [159]:
def convert_to_cbm_cost(cost, mode, product):
    kg_orderlines = demand_data.loc[demand_data['Product'] == product, 'Kg/Orderlines'].values[0]
    cbm_orderlines = demand_data.loc[demand_data['Product'] == product, 'CBM/Orderlines'].values[0]
    
    if mode == 'FCL20' or mode == 'FCL40':
        return (cost / kg_orderlines) * cbm_orderlines
    if mode == 'Air':
        return cost * cbm_orderlines
    return cost

In [160]:
products = demand_data['Product'].unique()
suppliers = demand_data['Supplier'].unique()
transportation_modes = transport_cost_data['Mode'].unique()

In [161]:
for p in products:
    for transport in transport_cost_data.values:
        print(f'{p} : {transport[4]: >8} --- {convert_to_cbm_cost(transport[4], transport[3], p)}')

A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :      LCL --- LCL
A-001 :      Air --- Air
A-001 :    FCL20 --- FCL20
A-001 :    FCL40 --- FCL40
A-001 :   

In [162]:
pivoted_transport_df = transport_cost_data.copy()

pivoted_transport_df['id'] = pivoted_transport_df.groupby('Supplier Country').cumcount()

# pivoted_transport_df
pivoted_transport_df = pivoted_transport_df.pivot_table(index='Supplier Country', columns='id', values='Cost').reset_index()
pivoted_transport_df = pivoted_transport_df.iloc[:, :5]

pivoted_transport_df.columns = ['Supplier Country', 'FCL20 Cost', 'FCL40 Cost', 'LCL Cost', 'Air Cost']

pivoted_transport_df
modes  = ['FCL20', 'FCL40', 'LCL', 'Air']

merged_demand_data = pd.merge(demand_data, pivoted_transport_df, on='Supplier Country')

for mode in modes:
    column_name = f'{mode} Cost'
    if column_name in merged_demand_data.columns:
        # Use pandas apply method with a lambda function
        merged_demand_data[column_name] = merged_demand_data.apply(
            lambda row: convert_to_cbm_cost(row[column_name], mode, row['Product']),
            axis=1
        )

merged_demand_data


,Product,Supplier,Supplier Country,Supplier City,Lead Time (Weeks),Mean Weekly Demand,Total_Sales,Safety Stock (SS),SS (round unit up),SS (CBM),...,Order-Up-To Level (S),S (round unit up),S (CBM),S (KG),CBM/Orderlines,Kg/Orderlines,FCL20 Cost,FCL40 Cost,LCL Cost,Air Cost
0,A-001,S-CN-001,China,Dongguan,5.857143,0.423077,33,0.726923,1,0.575875,...,5.682967,6,3.455248,1058.763708,0.575875,176.460618,22.016738,35.654961,242.68,6.789563
1,A-002,S-CN-001,China,Dongguan,5.857143,12.794872,1070,25.505128,26,3.380420,...,175.387912,176,22.882846,3987.955801,0.130016,22.658840,38.710826,62.690169,242.68,1.532891
2,A-003,S-CN-002,China,Shenzhen,5.857143,8.705128,683,18.444872,19,3.022785,...,120.419231,121,19.250366,1417.300274,0.159094,11.713225,91.632567,148.394175,242.68,1.875717
3,A-004,S-DE-001,Germany,Mainz,12.714286,566.076923,44194,142.123077,143,0.759798,...,14536.650550,14537,77.239021,23915.387590,0.005313,1.645139,26.985743,42.390342,597.87,0.060784
4,A-005,S-CN-003,China,Changzhou,5.857143,323.076923,25159,184.873077,185,19.171669,...,3969.488462,3970,411.413649,73316.907250,0.103631,18.467735,37.857095,61.307596,242.68,1.221805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,B-039,S-DE-004,Germany,Munich,12.714286,0.282051,22,0.867949,1,1.015196,...,8.040110,9,9.136765,6244.055282,1.015196,693.783920,12.226461,19.205840,597.87,11.613843
79,B-040,S-US-003,USA,Phoenix,6.857143,0.051282,5,-0.051282,-1,-0.228345,...,0.652015,1,0.228345,82.397518,0.228345,82.397518,29.580286,44.214005,893.90,2.194395
80,B-041,S-DE-005,Germany,Frankfurt,12.714286,0.012821,1,-0.012821,-1,-0.050189,...,0.313187,1,0.050189,11.424678,0.050189,11.424678,36.705967,57.659280,597.87,0.574158
81,B-042,S-DE-004,Germany,Munich,12.714286,0.576923,45,1.423077,2,1.201716,...,16.093407,17,10.214587,5330.137991,0.600858,313.537529,16.012436,25.153009,597.87,6.873816
